## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\Carmen\Documents\analytics_bootcamp\api\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kaitangata,NZ,-46.2817,169.8464,73.99,35,54,17.00,broken clouds
1,1,Takoradi,GH,4.8845,-1.7554,79.83,88,63,9.98,broken clouds
2,2,Callaway,US,30.1530,-85.5699,59.00,64,31,3.00,scattered clouds
3,3,Luderitz,NaN,-26.6481,15.1594,69.93,54,0,15.23,clear sky
4,4,Torbay,CA,47.6666,-52.7314,37.00,100,90,13.80,fog
5,5,Diffa,NE,13.3154,12.6113,76.06,14,9,9.10,clear sky
6,6,New Norfolk,AU,-42.7826,147.0587,64.00,48,19,5.01,few clouds
7,7,Namibe,AO,-15.1961,12.1522,77.43,82,42,4.12,scattered clouds
8,8,Mar Del Plata,AR,-38.0023,-57.5575,68.00,88,0,8.05,light rain
9,9,Norman Wells,CA,65.2820,-126.8329,12.20,57,75,6.91,broken clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Takoradi,GH,4.8845,-1.7554,79.83,88,63,9.98,broken clouds
5,5,Diffa,NE,13.3154,12.6113,76.06,14,9,9.10,clear sky
7,7,Namibe,AO,-15.1961,12.1522,77.43,82,42,4.12,scattered clouds
10,10,Ouidah,BJ,6.3631,2.0851,84.20,84,20,11.50,few clouds
16,16,Arraial Do Cabo,BR,-22.9661,-42.0278,76.17,74,13,8.12,few clouds
18,18,Rikitea,PF,-23.1203,-134.9692,77.32,65,26,17.36,scattered clouds
23,23,Atar,MR,20.5169,-13.0499,84.29,19,60,8.48,broken clouds
24,24,Kapaa,US,22.0752,-159.3190,78.80,65,75,19.57,light rain
25,25,Vaini,TO,-21.2000,-175.2000,77.00,78,90,9.22,light rain
27,27,Puerto Ayora,EC,-0.7393,-90.3518,77.00,94,38,7.00,scattered clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                218
City                   218
Country                218
Lat                    218
Lng                    218
Max Temp               218
Humidity               218
Cloudiness             218
Wind Speed             218
Current Description    218
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Takoradi,GH,79.83,broken clouds,4.8845,-1.7554,
5,Diffa,NE,76.06,clear sky,13.3154,12.6113,
7,Namibe,AO,77.43,scattered clouds,-15.1961,12.1522,
10,Ouidah,BJ,84.20,few clouds,6.3631,2.0851,
16,Arraial Do Cabo,BR,76.17,few clouds,-22.9661,-42.0278,
18,Rikitea,PF,77.32,scattered clouds,-23.1203,-134.9692,
23,Atar,MR,84.29,broken clouds,20.5169,-13.0499,
24,Kapaa,US,78.80,light rain,22.0752,-159.3190,
25,Vaini,TO,77.00,light rain,-21.2000,-175.2000,
27,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.    
    hotels = requests.get(base_url, params=params).json()    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Takoradi,GH,79.83,broken clouds,4.8845,-1.7554,Raybow International Hotel
5,Diffa,NE,76.06,clear sky,13.3154,12.6113,ProDAF/Diffa
7,Namibe,AO,77.43,scattered clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
10,Ouidah,BJ,84.20,few clouds,6.3631,2.0851,Djegba Hotel
16,Arraial Do Cabo,BR,76.17,few clouds,-22.9661,-42.0278,Pousada Porto Praia
18,Rikitea,PF,77.32,scattered clouds,-23.1203,-134.9692,Pension Maro'i
23,Atar,MR,84.29,broken clouds,20.5169,-13.0499,Oumou Elghoura
24,Kapaa,US,78.80,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
25,Vaini,TO,77.00,light rain,-21.2000,-175.2000,Keleti Beach Resort
27,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [23]:
hotel_df.count()

City                   218
Country                218
Max Temp               218
Current Description    218
Lat                    218
Lng                    218
Hotel Name             218
dtype: int64

In [24]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [9]:
# 7. Drop the rows where there is no Hotel Name.


In [25]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))